### (naive) 접근
- 왼쪽으로 돌릴때만 이후 결과에 영향을 줄 것이다.
- 한쪽방향으로 돌리기로 정했으면, 회전 도중에 회전방향을 바꿀 필요는 없다. 
  - 즉, 왼쪽방향이나, 오른쪽 방향으로만 돌리는 경우를 생각하면 된다.

In [ ]:
### Naive(O(2^N))
N = int(input())
A = [*map(int, input().strip())]
B = [*map(int, input().strip())]

acc_record = [0] * N 
answer = [float("inf")] * N 
def sol(i, acc, res) : #현재 숫자의 인덱스, 누적 회전수
  global answer, acc_record
  if i == N : return
  if (A[i] + acc) % 10 == B[i] : 
    if answer[i] > res :
      acc_record[i] = acc
      answer[i] = res
    sol(i+1, acc, res)
  x = (A[i] + acc) % 10 - B[i]

  if answer[i] > res + (10-x)%10 : #왼쪽(+방향)으로 돌리고 누적 회전도 그만큼 증가시키는 경우
    acc_record[i] = acc + (10-x)%10
    answer[i] = res + (10-x)%10
  sol(i+1, acc + (10-x)%10, res + (10-x)%10) 

  if answer[i] > res + (10+x)%10 :
    acc_record[i] = acc
    answer[i] = res + (10+x)%10
  sol(i+1, acc, res + (10+x)%10) #오른쪽(-방향)으로 돌리고 누적회전은 그대로인 경우

sol(0, 0, 0)
print(answer, acc_record)

- 매 칸마다 회전수는 그대로거나 증가한다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol():
  N = int(input())
  A = [*map(int, input().strip())]
  B = [*map(int, input().strip())]

  DP = [[float("inf")] * 10 for _ in range(N+1)]
  DP[0] = [*range(10)]

  for i, (a, b) in enumerate(zip(A, B)):
    for j in range(10) :
      l = (b - a - j) % 10 #(현재 상태에서 B에 맞추기 위해)왼쪽으로 회전해야 하는 횟수
      r = 10 - l #오른쪽으로 회전해야 하는 횟수
      DP[i+1][j] = min(DP[i+1][j], DP[i][j] + r) #오른쪽으로 돌린 경우
      DP[i+1][(j + l) % 10] = min(DP[i+1][(j + l) % 10], DP[i][j] + l) #왼쪽으로 돌린 경우

  print(min(DP[N]))

sol()

- N의 개수를 가지고 DP의 구조가 어떻게 될지 가늠을 해야한다. 
- 1차원 DP로 `O(N)`으로 돌릴 수 있으리란 희망을 가지고 삽질하는건 의미가 없다.

### 풀이
- `DP[i][j]` = `i`번째 원통에서 나사를 왼쪽으로 $\Nu \equiv j \pmod {10}$번 돌렸을 때의 최소 회전수
  - 이때 숫자나사는 원통이므로 j는 0~9까지만 고려하면 된다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol():
  N = int(input())
  A = [*map(int, input().strip())]
  B = [*map(int, input().strip())]

  DP = [[float("inf")] * 10 for _ in range(N+1)]
  DP[0] = [*range(10)]
  DP2 = [[None] * 10 for _ in range(N)] #회전량

  for i, (a, b) in enumerate(zip(A, B)):
    for j in range(10) :
      l = (b - a - j) % 10 #(현재 상태에서 B에 맞추기 위해)왼쪽으로 회전해야 하는 횟수
      r = 10 - l #오른쪽으로 회전해야 하는 횟수
      if DP[i+1][j] > DP[i][j] + r : #오른쪽으로 돌린 경우가 더 적은 경우
        DP[i+1][j] = DP[i][j] + r 
        DP2[i][j] = -r

      if DP[i+1][(j + l) % 10] > DP[i][j] + l : #왼쪽으로 돌린 경우가 더 적은 경우
        DP[i+1][(j + l) % 10] = DP[i][j] + l
        DP2[i][(j + l) % 10] = l

  answer = float("inf")
  for i, v in enumerate(DP[N]) :
    if answer > v :
      answer = min(answer, v)
      cur = i
  
  sys.stdout.write(str(answer) + "\n")
  answer = []
  for i in range(N-1, -1, -1) : #음수면 그대로 위칸, 양수면 -n칸후 위칸
    answer.append(DP2[i][cur])
    if DP2[i][cur] < 0 : continue
    cur = (cur - DP2[i][cur]) % 10
  
  for i, v in enumerate(answer[::-1]) :
    sys.stdout.write(str(i+1) + " " + str(v) + "\n")

sol()

### 2494
- 위 문제의 해를 역추적해서 출력하는 문제.
- `DP2[i][j]`를 정의하고, `DP[i][j]`의 최소 회전수를 갱신할 때 DP2에 얼만큼 회전했는지의 정보를 저장한다.
  - 이때 왼쪽으로 돌렸으면 양수를, 오른쪽으로 돌렸으면 음수를 저장하여 값만으로 얼만큼 돌렸는지를 알 수 있게 한다.
- 이후 맨 아랫줄부터 역추적하여 출력한다.